In [92]:
import sys
import argparse
import os
import pandas as pd
import datetime as dt
from datetime import timedelta
from datetime import datetime
import numpy as np
import json
import geopandas as gpd
from urllib.request import urlopen

In [93]:
world_cases = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
world_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/8/21,12/9/21,12/10/21,12/11/21,12/12/21,12/13/21,12/14/21,12/15/21,12/16/21,12/17/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,157542,157585,157858,157858,157858,157648,157660,157665,157725,157734
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,202641,202863,203215,203524,203787,203925,204301,204627,204928,205224
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,212047,212224,212434,212652,212848,213058,213288,213533,213745,214044
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,18815,19272,19440,19440,19440,19440,20136,20136,20549,20549
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,65332,65346,65371,65397,65404,65404,65431,65565,65648,65760


In [94]:
print(world_cases.columns)

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '12/8/21', '12/9/21', '12/10/21', '12/11/21', '12/12/21', '12/13/21',
       '12/14/21', '12/15/21', '12/16/21', '12/17/21'],
      dtype='object', length=700)


In [95]:
# changing the columns to match the US aggregated data
world_cases = world_cases.drop(['Province/State', 'Lat', 'Long'], axis = 1)

# format the dates
dic = {}
for i in range(len(world_cases.columns)):
    to_replace = list(world_cases.columns)[i].replace('/', '-')
    year = to_replace[-2:]  
    year = "20" + year
    to_replace = to_replace[:len(to_replace) - 3]
    
    to_replace = year + '-' + to_replace
    
    dic[list(world_cases.columns)[i]] = to_replace
    

world_cases = world_cases.rename(index = str, columns = dic)
world_cases = world_cases.rename(columns = {"20on-Country-Reg": "name"})
world_cases.head()

,name,2020-1-22,2020-1-23,2020-1-24,2020-1-25,2020-1-26,2020-1-27,2020-1-28,2020-1-29,2020-1-30,...,2021-12-8,2021-12-9,2021-12-10,2021-12-11,2021-12-12,2021-12-13,2021-12-14,2021-12-15,2021-12-16,2021-12-17
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,157542,157585,157858,157858,157858,157648,157660,157665,157725,157734
1,Albania,0,0,0,0,0,0,0,0,0,...,202641,202863,203215,203524,203787,203925,204301,204627,204928,205224
2,Algeria,0,0,0,0,0,0,0,0,0,...,212047,212224,212434,212652,212848,213058,213288,213533,213745,214044
3,Andorra,0,0,0,0,0,0,0,0,0,...,18815,19272,19440,19440,19440,19440,20136,20136,20549,20549
4,Angola,0,0,0,0,0,0,0,0,0,...,65332,65346,65371,65397,65404,65404,65431,65565,65648,65760


In [96]:
# filter out the dates not needed
worldStart = dt.datetime(2020, 1, 22)
startDate = dt.datetime(2020, 2, 14)
endDate = dt.datetime(2020, 4, 15)
# print(int((endDate - startDate).days))

# datetime_columns = pd.to_datetime(world_cases.columns[1:])
# column_drop_list = [i for i in list(world_case.columns) if date]


In [97]:
len(world_cases)

280

In [98]:
# group by name
world_cases = pd.DataFrame(world_cases.groupby(by = ["name"], as_index = False).sum())
world_cases.head()

,name,2020-1-22,2020-1-23,2020-1-24,2020-1-25,2020-1-26,2020-1-27,2020-1-28,2020-1-29,2020-1-30,...,2021-12-8,2021-12-9,2021-12-10,2021-12-11,2021-12-12,2021-12-13,2021-12-14,2021-12-15,2021-12-16,2021-12-17
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,157542,157585,157858,157858,157858,157648,157660,157665,157725,157734
1,Albania,0,0,0,0,0,0,0,0,0,...,202641,202863,203215,203524,203787,203925,204301,204627,204928,205224
2,Algeria,0,0,0,0,0,0,0,0,0,...,212047,212224,212434,212652,212848,213058,213288,213533,213745,214044
3,Andorra,0,0,0,0,0,0,0,0,0,...,18815,19272,19440,19440,19440,19440,20136,20136,20549,20549
4,Angola,0,0,0,0,0,0,0,0,0,...,65332,65346,65371,65397,65404,65404,65431,65565,65648,65760


In [99]:
len(world_cases)

196

In [100]:
# convert cumulative to daily
for i in range(len(world_cases.columns) - 1, 1, -1):
    world_cases[world_cases.columns[i]] = world_cases[world_cases.columns[i]].sub(world_cases[world_cases.columns[i - 1]], axis = 0)
# world_cases.head()

In [101]:
world_cases = world_cases.drop(world_cases.columns[1:(1+int((startDate - worldStart).days))], axis = 1)
world_cases = world_cases.drop(world_cases.columns[(1+int((endDate - startDate).days)):], axis = 1)
world_cases.head()

,name,2020-2-14,2020-2-15,2020-2-16,2020-2-17,2020-2-18,2020-2-19,2020-2-20,2020-2-21,2020-2-22,...,2020-4-5,2020-4-6,2020-4-7,2020-4-8,2020-4-9,2020-4-10,2020-4-11,2020-4-12,2020-4-13,2020-4-14
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,29,38,30,56,21,77,0,34,52,58
1,Albania,0,0,0,0,0,0,0,0,0,...,28,16,6,17,9,7,17,13,21,8
2,Algeria,0,0,0,0,0,0,0,0,0,...,69,103,45,104,94,95,64,89,69,87
3,Andorra,0,0,0,0,0,0,0,0,0,...,35,24,20,19,19,18,0,37,8,13
4,Angola,0,0,0,0,0,0,0,0,0,...,4,2,1,2,0,0,0,0,0,0


In [102]:
# merge with the data with geoid
world_geoid = pd.read_csv('data/georegion.csv')
print(world_geoid.columns)
world_cases = world_geoid.merge(world_cases, left_on = world_geoid.columns[0], right_on = "name", how = 'left')


# to_drop = []
# to_drop.append(len(world_cases.columns) - 1)
# for i in range(2, 25):
#     to_drop.append(i)
# world_cases = world_cases.drop(world_cases.columns[to_drop], axis = 1)

world_cases = world_cases.drop(world_cases.columns[2], axis = 1)
world_cases.head()

Index(['﻿geoName', 'geoRegion'], dtype='object')


,﻿geoName,geoRegion,2020-2-14,2020-2-15,2020-2-16,2020-2-17,2020-2-18,2020-2-19,2020-2-20,2020-2-21,...,2020-4-5,2020-4-6,2020-4-7,2020-4-8,2020-4-9,2020-4-10,2020-4-11,2020-4-12,2020-4-13,2020-4-14
0,Antigua and Barbuda,NORTH AMERICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0
1,Algeria,AFRICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,69.0,103.0,45.0,104.0,94.0,95.0,64.0,89.0,69.0,87.0
2,Azerbaijan,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,63.0,57.0,76.0,105.0,104.0,65.0,67.0,40.0,50.0,49.0
3,Albania,EURPOE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28.0,16.0,6.0,17.0,9.0,7.0,17.0,13.0,21.0,8.0
4,Armenia,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,52.0,11.0,20.0,28.0,40.0,16.0,30.0,46.0,26.0,28.0


In [103]:
len(world_cases)

238

In [104]:
# world_cases[world_cases['_merge'] != 'both']

In [105]:
len(world_cases)

238

In [106]:
output_cases = pd.read_csv("data/output_cases.csv")
output_cases.head()


# world_cases
# world_cases.to_csv("data/world_cases.csv", index = False)

,name,state,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,...,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14
0,Phoenix-Mesa-Scottsdale,AZ,1,0,0,0,0,0,0,0,...,169,111,75,68,143,53,158,86,65,43
1,San Jose-Sunnyvale-Santa Clara,CA,2,0,0,0,0,0,0,0,...,59,24,62,97,63,43,82,55,48,1
2,San Francisco-Oakland-Hayward,CA,2,0,0,0,0,0,0,0,...,219,94,122,146,130,137,154,72,196,126
3,San Diego-Carlsbad,CA,1,0,0,0,0,0,0,0,...,117,78,50,76,98,68,65,43,43,83
4,Los Angeles-Long Beach-Anaheim,CA,2,0,0,0,0,0,0,0,...,711,468,599,705,488,534,526,375,234,643


In [107]:
# world_cases = pd.concat([output_cases, world_cases], axis = 0)
world_cases.columns


Index(['﻿geoName', 'geoRegion', '2020-2-14', '2020-2-15', '2020-2-16',
       '2020-2-17', '2020-2-18', '2020-2-19', '2020-2-20', '2020-2-21',
       '2020-2-22', '2020-2-23', '2020-2-24', '2020-2-25', '2020-2-26',
       '2020-2-27', '2020-2-28', '2020-2-29', '2020-3-1', '2020-3-2',
       '2020-3-3', '2020-3-4', '2020-3-5', '2020-3-6', '2020-3-7', '2020-3-8',
       '2020-3-9', '2020-3-10', '2020-3-11', '2020-3-12', '2020-3-13',
       '2020-3-14', '2020-3-15', '2020-3-16', '2020-3-17', '2020-3-18',
       '2020-3-19', '2020-3-20', '2020-3-21', '2020-3-22', '2020-3-23',
       '2020-3-24', '2020-3-25', '2020-3-26', '2020-3-27', '2020-3-28',
       '2020-3-29', '2020-3-30', '2020-3-31', '2020-4-1', '2020-4-2',
       '2020-4-3', '2020-4-4', '2020-4-5', '2020-4-6', '2020-4-7', '2020-4-8',
       '2020-4-9', '2020-4-10', '2020-4-11', '2020-4-12', '2020-4-13',
       '2020-4-14'],
      dtype='object')

In [108]:
output_cases.columns

Index(['name', 'state', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17',
       '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22',
       '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27',
       '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03',
       '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08',
       '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
       '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18',
       '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23',
       '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28',
       '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02',
       '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06', '2020-04-07',
       '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11', '2020-04-12',
       '2020-04-13', '2020-04-14'],
      dtype='object')

In [109]:
rename_col = {}
for i in range(len(output_cases.columns)):
    rename_col[list(output_cases.columns)[i]] = list(world_cases.columns)[i]
output_cases = output_cases.rename(index = str, columns = rename_col)
# print(rename_col)
# print(output_cases.columns == world_cases.columns)

In [110]:
world_cases = world_cases.append(output_cases, ignore_index = True)

In [111]:
world_cases.head()

,﻿geoName,geoRegion,2020-2-14,2020-2-15,2020-2-16,2020-2-17,2020-2-18,2020-2-19,2020-2-20,2020-2-21,...,2020-4-5,2020-4-6,2020-4-7,2020-4-8,2020-4-9,2020-4-10,2020-4-11,2020-4-12,2020-4-13,2020-4-14
0,Antigua and Barbuda,NORTH AMERICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0
1,Algeria,AFRICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,69.0,103.0,45.0,104.0,94.0,95.0,64.0,89.0,69.0,87.0
2,Azerbaijan,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,63.0,57.0,76.0,105.0,104.0,65.0,67.0,40.0,50.0,49.0
3,Albania,EURPOE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28.0,16.0,6.0,17.0,9.0,7.0,17.0,13.0,21.0,8.0
4,Armenia,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,52.0,11.0,20.0,28.0,40.0,16.0,30.0,46.0,26.0,28.0


In [112]:
georegion_af = pd.read_csv("./data/georegion_af.csv")

In [113]:
georegion_af = georegion_af[georegion_af['GEOID'] != -9999]

In [114]:
georegion_af.head()

,GEOID,geoName,geoRegion,geoLON,geoLAT,geoISO3
1,1,Antigua and Barbuda,NORTH AMERICA,-61.791,17.280,ATG
2,2,Algeria,AFRICA,2.632,28.163,DZA
3,3,Azerbaijan,ASIA,47.532,40.292,AZE
4,4,Albania,EURPOE,20.068,41.143,ALB
5,5,Armenia,ASIA,44.948,40.286,ARM


In [115]:
len(georegion_af)

1168

In [116]:
world_cases.columns

Index(['﻿geoName', 'geoRegion', '2020-2-14', '2020-2-15', '2020-2-16',
       '2020-2-17', '2020-2-18', '2020-2-19', '2020-2-20', '2020-2-21',
       '2020-2-22', '2020-2-23', '2020-2-24', '2020-2-25', '2020-2-26',
       '2020-2-27', '2020-2-28', '2020-2-29', '2020-3-1', '2020-3-2',
       '2020-3-3', '2020-3-4', '2020-3-5', '2020-3-6', '2020-3-7', '2020-3-8',
       '2020-3-9', '2020-3-10', '2020-3-11', '2020-3-12', '2020-3-13',
       '2020-3-14', '2020-3-15', '2020-3-16', '2020-3-17', '2020-3-18',
       '2020-3-19', '2020-3-20', '2020-3-21', '2020-3-22', '2020-3-23',
       '2020-3-24', '2020-3-25', '2020-3-26', '2020-3-27', '2020-3-28',
       '2020-3-29', '2020-3-30', '2020-3-31', '2020-4-1', '2020-4-2',
       '2020-4-3', '2020-4-4', '2020-4-5', '2020-4-6', '2020-4-7', '2020-4-8',
       '2020-4-9', '2020-4-10', '2020-4-11', '2020-4-12', '2020-4-13',
       '2020-4-14'],
      dtype='object')

In [117]:
world_cases = world_cases.merge(georegion_af, left_on = [world_cases.columns[0], "geoRegion"] , right_on = ["geoName", "geoRegion"], how = 'right')

In [118]:
# reorder the columns
columns_reordered1 = list(world_cases.columns)[1:2]
columns_reordered2 = list(world_cases.columns)[2:-6]
columns_reordered3 = list(world_cases.columns)[-5:-3]
columns_reordered4 = list(world_cases.columns)[-3:]
columns_reordered = columns_reordered3 + columns_reordered1 + columns_reordered4 + columns_reordered2 


world_cases = world_cases.reindex(columns = columns_reordered)

In [119]:
# change the floats to integers
def float_to_int(x):
    if (x != x):
         # print("hey")
        return -9999
    x = int(x)
    return x

for i in list(world_cases.columns[6:]):
    world_cases[i] = world_cases[i].apply(float_to_int)

def format_float(x):
    x = round(x, 3)
    return x

world_cases['geoLON'] = world_cases['geoLON'].apply(format_float)
world_cases['geoLAT'] = world_cases['geoLAT'].apply(format_float)
    

In [120]:
world_cases.head()

,GEOID,geoName,geoRegion,geoLON,geoLAT,geoISO3,2020-2-14,2020-2-15,2020-2-16,2020-2-17,...,2020-4-4,2020-4-5,2020-4-6,2020-4-7,2020-4-8,2020-4-9,2020-4-10,2020-4-11,2020-4-12,2020-4-13
0,1,Antigua and Barbuda,NORTH AMERICA,-61.791,17.280,ATG,0,0,0,0,...,0,0,0,4,0,0,0,2,0,2
1,2,Algeria,AFRICA,2.632,28.163,DZA,0,0,0,0,...,80,69,103,45,104,94,95,64,89,69
2,3,Azerbaijan,ASIA,47.532,40.292,AZE,0,0,0,0,...,78,63,57,76,105,104,65,67,40,50
3,4,Albania,EURPOE,20.068,41.143,ALB,0,0,0,0,...,29,28,16,6,17,9,7,17,13,21
4,5,Armenia,ASIA,44.948,40.286,ARM,0,0,0,0,...,34,52,11,20,28,40,16,30,46,26


In [121]:
len(world_cases)

1168

In [122]:
world_cases.to_csv("data/world_cases.csv", index = False)